In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os


In [2]:
#Load actigraphy data of all participants to dataframes

import os
train = pd.read_csv('./train.csv')
acti_dfs = []

list_all_ids = [path.split('=')[1] for path in os.listdir('./series_train.parquet')]

for id in list_all_ids:
    serie_df = pd.read_parquet(f'./series_train.parquet/id={id}/part-0.parquet')
    # Create a continuous time scale in days by transforming the `time_of_day` column (which is in nanoseconds) to hours and then combining it with `relative_date_PCIAT``
    serie_df['time_of_day_hours'] = (
        serie_df['time_of_day'] / 1e9 / 3600 #nanoseconds to hours
    )
    serie_df['day_time'] = serie_df['relative_date_PCIAT'] + (
        serie_df['time_of_day_hours'] / 24
    )
    serie_df['time_periods_in_day'] = serie_df['time_of_day_hours'].apply(lambda x: round(x))
    serie_df.drop(columns=['time_of_day'], inplace=True)
    # serie_df = filter_actigraphy_data(serie_df)
    acti_dfs.append(serie_df)



KeyboardInterrupt: 

In [3]:
# create all of the dataframe, lists of data that will be included in the final aggregated dataframe

stats_df = pd.DataFrame()
stats_df['id'] = list_all_ids

bed_time_stats = []
wake_time_stats = []
avg_sleep_hour_stats = []
PCIAT_relevancy = []
low_enmo_percentage = []
mod_enmo_percentage = []
high_enmo_percentage = []
non_active_time_percentage = []
total_step_recorded = []
low_light_percentage_percentage = []
mod_light_percentage_percentage = []
high_light_percentage_percentage = []
quarter = []





In [ ]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df['time_of_day_hours'] = (
        df['time_of_day'] / 1e9 / 3600 #nanoseconds to hours
    )
    df['day_time'] = df['relative_date_PCIAT'] + (
        df['time_of_day_hours'] / 24
    )
    df['time_periods_in_day'] = df['time_of_day_hours'].apply(lambda x: round(x))
    df.drop(columns=['time_of_day'], inplace=True)
    description = {
        'PCIAT_relevancy': 1 / (0.5 + abs(df['day_time'].mean())) + 1,
        'low_enmo_percentage': (df['enmo'] < 0.1).sum() / len(df) * 100,
        'mod_enmo_percentage': ((df['enmo'] >= 0.1) & (df['enmo'] < 0.3)).sum() / len(df) * 100,
        'high_enmo_percentage': (df['enmo'] >= 0.3).sum() / len(df) * 100,
        'non_active_time_percentage': (df['enmo'] < 0.01).sum() / len(df) * 100,
        'total_step_recorded': len(df),
        'low_light_percentage_percentage': (df['light'] <= 30).sum() / len(df) * 100,
        'mod_light_percentage_percentage': ((df['light'] > 30) & (df['light'] <= 100)).sum() / len(df) * 100,
        'high_light_percentage_percentage': (df['light'] > 100).sum() / len(df) * 100,
        'quarter': df['quarter'].mode()[0],
    }

    average_enmo_each_hour = df.groupby('time_periods_in_day')['enmo'].mean()
    average_enmo_each_hour = average_enmo_each_hour.reset_index()
    average_enmo_each_hour['enmo'] = average_enmo_each_hour['enmo'].rolling(window=3, min_periods=1).mean()
    average_enmo_each_hour['enmo'] = average_enmo_each_hour['enmo'] / average_enmo_each_hour['enmo'].max()
    for i in range(24):
        try:
            hour_i_enmo = average_enmo_each_hour[average_enmo_each_hour['time_periods_in_day'] == i]['enmo'].values[0]
        except:
            hour_i_enmo = 0
        description[f'enmo_rate_hour_{i}'] = hour_i_enmo
    # actigraphy_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light']
    # df = df[df['non-wear_flag'] != 1][actigraphy_cols]
    # df.drop('step', axis=1, inplace=True)
    return list(description.values()), filename.split('=')[1]

### Feature engineering part before filtering

In [4]:
# Calculate average enmo rate for each hour
enmo_rate_hourly = {}
for i in range(24):
    enmo_rate_hourly[i] = []

for df in acti_dfs:
    average_enmo_each_hour = df.groupby('time_periods_in_day')['enmo'].mean()
    average_enmo_each_hour = average_enmo_each_hour.reset_index()
    average_enmo_each_hour['enmo'] = average_enmo_each_hour['enmo'].rolling(window=3, min_periods=1).mean()
    average_enmo_each_hour['enmo'] = average_enmo_each_hour['enmo'] / average_enmo_each_hour['enmo'].max()

    for i in range(24):
        try:
            hour_i_enmo = average_enmo_each_hour[average_enmo_each_hour['time_periods_in_day'] == i]['enmo'].values[0]
        except:
            hour_i_enmo = 0
        enmo_rate_hourly[i].append(hour_i_enmo)

        
for i in range(24):
    stats_df[f'enmo_rate_hour_{i}'] = enmo_rate_hourly[i]

In [5]:
stats_df

,id,enmo_rate_hour_0,enmo_rate_hour_1,enmo_rate_hour_2,enmo_rate_hour_3,enmo_rate_hour_4,enmo_rate_hour_5,enmo_rate_hour_6,enmo_rate_hour_7,enmo_rate_hour_8,...,enmo_rate_hour_14,enmo_rate_hour_15,enmo_rate_hour_16,enmo_rate_hour_17,enmo_rate_hour_18,enmo_rate_hour_19,enmo_rate_hour_20,enmo_rate_hour_21,enmo_rate_hour_22,enmo_rate_hour_23
0,0d01bbf2,0.297864,0.283888,0.261138,0.245731,0.247550,0.254277,0.256418,0.322703,0.522266,...,0.885505,1.000000,0.935717,0.638574,0.587933,0.576135,0.632850,0.568329,0.559607,0.419666
1,cefdb7fe,0.321726,0.241055,0.204247,0.134108,0.114444,0.105917,0.121156,0.199060,0.330032,...,0.867110,0.973346,1.000000,0.994802,0.979068,0.929680,0.896223,0.799960,0.694754,0.590830
2,58391429,0.499658,0.548132,0.551932,0.560905,0.527222,0.498281,0.522093,0.614756,0.678073,...,0.842698,0.849677,0.873659,0.983658,0.971107,0.836263,0.751661,0.952759,1.000000,0.835100
3,2ca2206f,0.202046,0.216466,0.219833,0.229158,0.228869,0.239768,0.280941,0.348548,0.429076,...,0.739251,0.816799,0.928419,1.000000,0.887322,0.768893,0.683936,0.608409,0.571509,0.456200
4,19455336,0.173444,0.169154,0.166165,0.163348,0.163626,0.171309,0.170389,0.190079,0.280574,...,0.913474,0.668506,0.613903,0.770542,0.800631,0.801412,0.582474,0.423959,0.295088,0.211030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,43a7386d,0.029891,0.030524,0.031445,0.032330,0.032630,0.032313,0.033179,0.052486,0.176663,...,0.400514,0.554785,0.521320,0.886873,1.000000,0.951540,0.624768,0.361108,0.319705,0.173030
992,2840643b,0.365415,0.260376,0.281409,0.188838,0.199570,0.229720,0.292861,0.381117,0.429046,...,0.707048,0.766533,0.880067,0.947458,1.000000,0.954656,0.909996,0.824607,0.667813,0.464196
993,1b329556,0.050363,0.053583,0.052782,0.049866,0.047034,0.045439,0.045556,0.146936,0.399889,...,0.864454,0.897309,0.777823,0.799528,0.888644,1.000000,0.806105,0.715645,0.532690,0.390610
994,62b873a2,0.058431,0.057075,0.052351,0.050466,0.054591,0.067384,0.090820,0.237819,0.315408,...,0.533361,0.680386,0.852610,1.000000,0.936404,0.727831,0.447653,0.263799,0.185252,0.102844


In [ ]:
import jiwer
     

def compute_cer(truth_and_output: list[tuple[str, str]]):
  """Computes CER given pairs of ground truth and model output."""
  class TokenizeTransform(jiwer.transforms.AbstractTransform):
    def process_string(self, s: str):
      return tokenize_expression(r'{}'.format(s))

    def process_list(self, tokens: list[str]):
      return [self.process_string(token) for token in tokens]

  ground_truth, model_output = zip(*truth_and_output)

  return jiwer.cer(truth=list(ground_truth),
            hypothesis=list(model_output),
            reference_transform=TokenizeTransform(),
            hypothesis_transform=TokenizeTransform(),
      )
     

# Test data to run compute_cer().
# The first element is the model prediction, the second the ground truth.
examples = [
    (r'\sqrt{2}', r'\sqrt{2}'),  # 0 mistakes, 4 tokens
    (r'\frac{1}{2}', r'\frac{i}{2}'),  # 1 mistake, 7 tokens
    (r'\alpha^{2}', 'a^{2}'),  # 1 mistake, 5 tokens
    ('abc', 'def'),  # 3 mistakes, 3 tokens
]

# 5 mistakes for 19 tokens: 26.3% error rate.
print(f"{compute_cer(examples)*100:.1f} %")

### Filtering: Cut out non-wear steps

In [6]:
def filter_non_wear_steps(df):
    # Filter out non-wear device periods
    df = df[df['non-wear_flag'] == 0]

    if df.shape[0] == 0:
        return None
    
    return df

In [7]:
for i in range(len(acti_dfs)):
    acti_dfs[i] = filter_non_wear_steps(acti_dfs[i])

### Filtering: Cut out periods with linear trends
There are parts in actigraphy data where X, Y, Z, enmo, light shows linear trends. For example, the enmo of a participant can be still for about 3 days even when the device shows that the participant is wearing it. This is very very unlikely to happen in real life.

### Feature engineering part after filtering

In [8]:
# calculate PCIAT relevancy

PCIAT_relevancy = []
for df in acti_dfs:
    PCIAT_relevancy.append(1 / (0.5 + abs(df['day_time'].mean())) + 1)

stats_df['PCIAT_relevancy'] = PCIAT_relevancy

In [9]:
# calculate low_enmo_percentage
low_enmo_percentage = []
for df in acti_dfs:
    low_enmo_percentage.append((df['enmo'] < 0.1).sum() / len(df) * 100)

stats_df['low_enmo_percentage'] = low_enmo_percentage

In [10]:
# calculate mod_enmo_percentage
mod_enmo_percentage = []
for df in acti_dfs:
    mod_enmo_percentage.append(((df['enmo'] >= 0.1) & (df['enmo'] < 0.3)).sum() / len(df) * 100)

stats_df['mod_enmo_percentage'] = mod_enmo_percentage

In [11]:
# calculate high_enmo_percentage
high_enmo_percentage = []
for df in acti_dfs:
    high_enmo_percentage.append((df['enmo'] >= 0.3).sum() / len(df) * 100)

stats_df['high_enmo_percentage'] = high_enmo_percentage

In [12]:
# calculate non_active_time_percentage
non_active_time_percentage = []
for df in acti_dfs:
    non_active_time_percentage.append((df['enmo'] < 0.01).sum() / len(df) * 100)

stats_df['non_active_time_percentage'] = non_active_time_percentage

In [13]:
# caculate total_step_recorded
total_step_recorded = []
for df in acti_dfs:
    total_step_recorded.append(len(df))

stats_df['total_step_recorded'] = total_step_recorded

In [14]:
# calculate low_light_percentage_percentage
low_light_percentage_percentage = []
for df in acti_dfs:
    low_light_percentage_percentage.append((df['light'] <= 30).sum() / len(df) * 100)

stats_df['low_light_percentage_percentage'] = low_light_percentage_percentage


In [15]:
# calculate mod_light_percentage_percentage
mod_light_percentage_percentage = []
for df in acti_dfs:
    mod_light_percentage_percentage.append(((df['light'] > 30) & (df['light'] <= 100)).sum() / len(df) * 100)

stats_df['mod_light_percentage_percentage'] = mod_light_percentage_percentage

In [16]:
# calculate high_light_percentage_percentage
high_light_percentage_percentage = []
for df in acti_dfs:
    high_light_percentage_percentage.append((df['light'] > 100).sum() / len(df) * 100)

stats_df['high_light_percentage_percentage'] = high_light_percentage_percentage

In [17]:
# calculate quarter
quarter = []
for df in acti_dfs:
    quarter.append(df['quarter'].mode()[0])

stats_df['quarter'] = quarter

In [21]:
# attach sii scores to the stats_df
sii = []
for id in list_all_ids:
    sii.append(train[train['id'] == id]['sii'].values[0])
    
stats_df['sii'] = sii

In [22]:
# save stats_df
stats_df.to_csv('stats_actigraphy.csv', index=False)
stats_df.head()

,id,enmo_rate_hour_0,enmo_rate_hour_1,enmo_rate_hour_2,enmo_rate_hour_3,enmo_rate_hour_4,enmo_rate_hour_5,enmo_rate_hour_6,enmo_rate_hour_7,enmo_rate_hour_8,...,low_enmo_percentage,mod_enmo_percentage,high_enmo_percentage,non_active_time_percentage,total_step_recorded,low_light_percentage_percentage,mod_light_percentage_percentage,high_light_percentage_percentage,quarter,sii
0,0d01bbf2,0.297864,0.283888,0.261138,0.245731,0.247550,0.254277,0.256418,0.322703,0.522266,...,83.218297,14.204986,2.576717,28.493512,269335,79.489112,13.763157,6.747731,1,0.0
1,cefdb7fe,0.321726,0.241055,0.204247,0.134108,0.114444,0.105917,0.121156,0.199060,0.330032,...,92.104702,7.471938,0.423360,53.244609,319114,79.303008,10.144651,10.552342,2,1.0
2,58391429,0.499658,0.548132,0.551932,0.560905,0.527222,0.498281,0.522093,0.614756,0.678073,...,99.132111,0.723829,0.144060,87.970312,141608,99.281820,0.228094,0.490085,1,2.0
3,2ca2206f,0.202046,0.216466,0.219833,0.229158,0.228869,0.239768,0.280941,0.348548,0.429076,...,88.142673,10.266734,1.590594,31.392266,311959,75.370802,11.489010,13.140188,3,1.0
4,19455336,0.173444,0.169154,0.166165,0.163348,0.163626,0.171309,0.170389,0.190079,0.280574,...,89.394026,8.140608,2.465366,43.349633,122497,94.177000,3.549475,2.273525,1,1.0
